In [1]:
import pickle
import pandas as pd

In [2]:
def metric_calculation(predictions, labels):
    sum_recall_1 = 0
    sum_precision_1 = 0
    num_test_data = len(predictions)
    # make sure that predictions and labels lists have the same length
    for i in range(len(predictions)):
        correct_prediction = 0
        actual_label = labels[i]
        # k = 1
        if predictions[i][0] in actual_label:
            correct_prediction += 1
        sum_precision_1 += (correct_prediction / 1)
        sum_recall_1 += (correct_prediction / len(actual_label))

    # Divide by numer of test data
    precision_1 = sum_precision_1 / num_test_data
    recall_1 = sum_recall_1 / num_test_data
    f1_1 = 2 * precision_1 * recall_1 / (precision_1 + recall_1)

    print("K = 1")
    print("P@1 = " + precision_1.__str__())
    print("R@1 = " + recall_1.__str__())
    print("F@1 = " + f1_1.__str__())

In [3]:
pickled_results = ["untuned_bertbase_results.pkl","untuned_distilbert_results.pkl", "finetuned_bertbase_results.pkl","finetuned_distilbert_results.pkl","cross_encoder.pkl", "query_expansion.pkl"]

In [4]:
for item in pickled_results:
    # Read embeddingsu thin
    results = pickle.load(open(f"./{item}", 'rb'))
    labels = results["labels"]
    predictions = results["predictions"]
    print(f"======== METRICS FOR {item} ============")
    metric_calculation(predictions,labels)

======== METRICS FOR untuned_bertbase_results.pkl ============
K = 1
P@1 = 0.32091016084739116
R@1 = 0.2613900876160584
F@1 = 0.28810818914169917
======== METRICS FOR untuned_distilbert_results.pkl ============
K = 1
P@1 = 0.3052177324440957
R@1 = 0.24765641054381732
F@1 = 0.27344063385898115
======== METRICS FOR finetuned_bertbase_results.pkl ============
K = 1
P@1 = 0.3464103570027462
R@1 = 0.2841870761643221
F@1 = 0.3122288208983802
======== METRICS FOR finetuned_distilbert_results.pkl ============
K = 1
P@1 = 0.33150255001961554
R@1 = 0.27078312690317397
F@1 = 0.2980821245137956
======== METRICS FOR cross_encoder.pkl ============
K = 1
P@1 = 0.40368772067477443
R@1 = 0.3343468026677128
F@1 = 0.36575985110440024
======== METRICS FOR query_expansion.pkl ============
K = 1
P@1 = 0.38681836014123183
R@1 = 0.3181639858768143
F@1 = 0.34914823603176287


# Qualitative results

In [104]:
test_df = pd.read_csv("../0.Datasets/train_test_split/test.csv")

In [105]:
test_df.head().drop(columns=["Unnamed: 0"])

,qid,docid,question,answer
0,7817,31330,Can you have a positive return with a balance below cost basis?,Have you owned the stock for longer than 2015? The stock appears to have grown in value since December 2014 from 72.85 to 73.5 which is about 0.89% growth in the year to date (2015).
1,6304,105557,Oversimplify it for me: the correct order of investing,Great questions -- the fact that you're thinking about it is what's most important. I think a priority should be maximizing any employer match in your 401(k) because it's free money. Second would be paying off high interest debt because it's a big expense. Everything else is a matter of setting good financial habits so I think the order of importance will vary from person to person. (That's wh...
2,7115,43508,Definition of “U.S. source” for US non-resident alien capital gains tax,The examples you provide in the question are completely irrelevant. It doesn't matter where the brokerage is or where is the company you own stocks in. For a fairly standard case of an non-resident alien international student living full time in the US - your capital gains are US sourced. Let me quote the following text a couple of paragraphs down the line you quoted on the same page: Gain or ...
3,5716,287327,Are car buying services worth it?,"I have used car buying services through Costco and USAA. Twice with a Ford, and once with a Honda. In all instances I was directed to sales people that were uncommonly friendly and pleasant to work with. I was given a deep discount without any negotiation. In two of the three cases I did not have a trade. In one case I had a trade, and negotiated a deeper discount then was originally offe..."
4,9016,580920,What makes a Company's Stock prices go up or down?,"Here are some significant factors affect the company stock price performance: Usually, profitability is known to the public through the financial statements; it won't be 100% accurate and people would also trade the stock with the price not matching to the true value of the firm. Still there are dozens of other various reasons exist. People are just not behaving as rational as what the textboo..."


In [106]:
question_answer_index_map = {}
for _,row in test_df.iterrows():
    if row["qid"] not in question_answer_index_map:
        question_answer_index_map[row["qid"]]= []
        question_answer_index_map[row["qid"]].append(row["docid"])
    else:
        question_answer_index_map[row["qid"]].append(row["docid"])

        
x = pickle.load(open("finetuned_bertbase_results.pkl", 'rb'))
finetuned_bertbase_predictions = x["predictions"]

# question_answer_index_map gives a dictionary where
# key: qid, value: list of docid
# e.g.    {qid: [docid1, docid2]}

# finetuned_bertbase_predictions give the fintuned bertbase's
# prediction for each qid in question_answer_index_map.
# same structure for the other pickle files

In [107]:
pd.set_option('max_colwidth',400)

In [142]:
test = test_df.loc[test_df['qid']==5716]
test[['question', 'answer']]

,question,answer
3,Are car buying services worth it?,"I have used car buying services through Costco and USAA. Twice with a Ford, and once with a Honda. In all instances I was directed to sales people that were uncommonly friendly and pleasant to work with. I was given a deep discount without any negotiation. In two of the three cases I did not have a trade. In one case I had a trade, and negotiated a deeper discount then was originally offe..."


In [143]:
docid = finetuned_bertbase_predictions[3][0]
ans = test_df.loc[test_df['docid']==docid]
ans[['answer']].rename(columns={'answer':'prediction'})

,prediction
3,"I have used car buying services through Costco and USAA. Twice with a Ford, and once with a Honda. In all instances I was directed to sales people that were uncommonly friendly and pleasant to work with. I was given a deep discount without any negotiation. In two of the three cases I did not have a trade. In one case I had a trade, and negotiated a deeper discount then was originally offe..."


In [89]:
finetuned_bertbase_predictions[:5]

[[28758], [250397], [43508], [287327], [596736]]

In [57]:
question_answer_index_map

{7817: [31330],
 6304: [105557, 244692, 250397],
 7115: [43508],
 5716: [287327],
 9016: [580920],
 2323: [130759],
 722: [349348, 370542],
 3466: [563959],
 9668: [13260, 111768],
 5025: [47904],
 776: [10440, 124027],
 10301: [288138],
 8522: [241175],
 2550: [81554, 181113, 48332, 459177, 34939],
 2636: [481692],
 5874: [547533],
 4514: [426270],
 4946: [121690],
 6692: [358129, 104916],
 3385: [435940, 104492],
 10456: [204943, 175200],
 10144: [477552],
 2069: [520087],
 5159: [514793],
 7723: [280626],
 2197: [313134, 426954, 34432],
 10979: [362762, 164001],
 5206: [201982, 28230],
 2787: [488326, 31477, 272695],
 10589: [445549],
 2717: [37960, 261345, 437871],
 762: [71338],
 9956: [149420, 190619],
 7802: [551145, 267998],
 4731: [480924, 171409],
 3857: [301833],
 4813: [98356],
 10481: [195824],
 5351: [325296],
 9841: [157414, 323475],
 8681: [261640],
 10497: [31483, 34913],
 4826: [54225],
 723: [561123, 261016],
 784: [203446],
 5894: [535793, 511159],
 1961: [282286],


In [26]:
len(question_answer_index_map)

2549

In [27]:
question_answer_index_map

{7817: [31330],
 6304: [105557, 244692, 250397],
 7115: [43508],
 5716: [287327],
 9016: [580920],
 2323: [130759],
 722: [349348, 370542],
 3466: [563959],
 9668: [13260, 111768],
 5025: [47904],
 776: [10440, 124027],
 10301: [288138],
 8522: [241175],
 2550: [81554, 181113, 48332, 459177, 34939],
 2636: [481692],
 5874: [547533],
 4514: [426270],
 4946: [121690],
 6692: [358129, 104916],
 3385: [435940, 104492],
 10456: [204943, 175200],
 10144: [477552],
 2069: [520087],
 5159: [514793],
 7723: [280626],
 2197: [313134, 426954, 34432],
 10979: [362762, 164001],
 5206: [201982, 28230],
 2787: [488326, 31477, 272695],
 10589: [445549],
 2717: [37960, 261345, 437871],
 762: [71338],
 9956: [149420, 190619],
 7802: [551145, 267998],
 4731: [480924, 171409],
 3857: [301833],
 4813: [98356],
 10481: [195824],
 5351: [325296],
 9841: [157414, 323475],
 8681: [261640],
 10497: [31483, 34913],
 4826: [54225],
 723: [561123, 261016],
 784: [203446],
 5894: [535793, 511159],
 1961: [282286],


# Evaluation of SAP CAI Bot

In [8]:
# importing libraries
import requests
import json
import os
import pickle
import pandas as pd

#!pip install aiohttp
import aiohttp
import asyncio
import time

# setting REST API parameters
clientId = "sb-5ef56e02-73a0-425d-add6-ae76de56d7a6-CLONE-BS-RT!b40741|cai-production!b20881"
secret = "binding-5ef56e02-73a0-425d-add6-ae76de56d7a6$L_YFqjgLUUPH9BQObHVnnNCmzyZFKiXSHDAv6TZhVuM="
requestToken = "f7df95de81789c3221d8bac32696a920"
oauthURL = "https://sapcai-community.authentication.eu10.hana.ondemand.com/oauth/token"
requestURL = "https://api.cai.tools.sap/build/v1/dialog"


In [5]:
# get sap cai evaluation dataset
sap_df = pd.read_csv("../0.Datasets/SAP/sap-qna-full-v2.csv", encoding='utf-8')
sample_qns = sap_df['question'].tolist()[:10]
sample_qns

['Claiming business expenses for a business with no income',
 'Transferring money from One business checking to another business checking',
 'Having a separate bank account for business/investing, but not a “business account?”',
 'Business Expense - Car Insurance Deductible For Accident That Occurred During a Business Trip',
 'Starting a new online business',
 '“Business day” and “due date” for bills',
 'New business owner - How do taxes work for the business vs individual?',
 'How to deposit a cheque issued to an associate in my business into my business account?',
 'Hobby vs. Business',
 'Personal checks instead of business ones']

## Testing regular API call speed for generating predictions

In [6]:
# functions to get response of the SAP CAI bot
def get_token():
    result = requests.post(oauthURL, data={'grant_type': 'client_credentials'}, auth=(clientId, secret))
    token = json.loads(result.content)
    return token["access_token"]

def get_response(user_input):
    # Make Post request, with text and credentials
    payload = json.dumps({
        "message": {
            "content": user_input,
            "type": "text"
        },
        "conversation_id": "anything_goes_here_11111"
    })
    result = requests.post(requestURL, data=payload,
                           headers={"Authorization": "Bearer " +
                                    get_token(), "X-Token": "Token " + requestToken,
                                    "Content-Type": "application/json"}
                           )
    # print(result)
    resultsjson = json.loads(result.content)
    return resultsjson['results']['messages'][0]['content']

In [7]:
# testing get_response -> average 8s per prediction
user_input = "should I get a roboadvisor or a financial advisor?"
get_response(user_input)

'**Advice for college student: Should I hire a financial adviser or just invest in index funds?**\nExactly what you do with the money depends on various personal choices you\'ll have to make for yourself.  Investing your money in Vanguard index funds such as the ones you mentioned is certainly one smart move.  However, I think you\'re quite right to be suspicious of an advisor with a 1% fee.  In many cases, such advisors are not worth their costs.  The thing to remember is that, typically with that type of fee structure, you always pay the costs, even if the advisor turns out to be wrong and your money doesn\'t grow. One thing to check is whether the advisor you mentioned is paid only by the fees he charges (a "fee-only financial planner\') , or whether he also makes money via the sales of financial products.  Some advisors earn money by selling you financial products (such as mutual funds), which can create a conflict of interest.  You can read about fee-only financial advisors and ch

In [9]:
start = time.time()
sample_pred = []
for qn in sample_qns:
    sample_pred.append(get_response(qn))
print("Time taken for regular API call on 10 inputs:", time.time()- start)


Time taken for regular API call on 10 inputs: 76.57765197753906


## Testing async API call speed for generating predictions

In [11]:
start_time = time.time()
async def main():
    # get all responses
    async with aiohttp.ClientSession() as session:
        predictions = []
        for qn in sample_qns:
            payload = payload = json.dumps({
                "message": {
                    "content": user_input,
                    "type": "text"
                },
                "conversation_id": "anything_goes_here_11111"
            })
            prediction = asyncio.ensure_future(
                get_async_response(session, payload))
            predictions.append(prediction)

        all_predictions = await asyncio.gather(*predictions)

    print('Number of videos:', len(all_predictions))

# get single response
async def get_async_response(session, payload):

    async with session.post(requestURL, data=payload,
                            headers={"Authorization": "Bearer " +
                                     bearer, "X-Token": "Token " + requestToken,
                                     "Content-Type": "application/json"}
                            ) as response:
        result_data = await response.content.json()
        print(result_data)
        message = result_data['results']['messages'][0]['content']
        return message

bearer = get_token()
asyncio.run(main())

print("--- %s seconds ---" % (time.time() - start_time))


RuntimeError: asyncio.run() cannot be called from a running event loop

In [21]:
#!pip3 install pickle5
import pickle5 as pickle # if you have python version <3.8

# import pickle
t = pickle.load(open(f"untuned_bertbase_results.pkl", 'rb'))
t['predictions']

[[280676],
 [160922],
 [43508],
 [494706],
 [596736],
 [87386],
 [454537],
 [563959],
 [111768],
 [263390],
 [124027],
 [288138],
 [241175],
 [138102],
 [481692],
 [349607],
 [240894],
 [121690],
 [104916],
 [185123],
 [340113],
 [81554],
 [520087],
 [518273],
 [62552],
 [426954],
 [362762],
 [69938],
 [272695],
 [413015],
 [222703],
 [205719],
 [515579],
 [34746],
 [374518],
 [301833],
 [98356],
 [195824],
 [357280],
 [352339],
 [261640],
 [449387],
 [562259],
 [261016],
 [107584],
 [511159],
 [282286],
 [278984],
 [329662],
 [133380],
 [524612],
 [52185],
 [561988],
 [509318],
 [40424],
 [396180],
 [321108],
 [397340],
 [179359],
 [132],
 [424641],
 [183166],
 [98356],
 [527076],
 [535110],
 [81554],
 [596537],
 [465542],
 [557921],
 [492456],
 [273789],
 [443511],
 [99336],
 [592780],
 [364594],
 [401125],
 [417301],
 [158058],
 [298551],
 [337561],
 [96268],
 [135765],
 [473647],
 [34139],
 [505213],
 [229271],
 [181113],
 [117451],
 [483218],
 [594475],
 [8018],
 [97842],
 [32057]